In [2]:
import os
from tensorflow import keras
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

import tensorflow_hub as hub
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from keras.models import load_model,save_model
import numpy as np
import urllib.request, urllib.error, json

In [3]:
class roopre:
    IMAGESIZE=(224,224)
    INPUTLAYERSIZE=IMAGESIZE + (3,)
    
    #def __init__(self):
        #print("MLFORKIDS: Downloading information about your machine learning project")
        #f = open("info.txt","r")
        #print(json.loads(f.read()))
        #self.__downloaded_training_images_list = json.loads(f.read())
        #self.scratchkey = scratchkey
    def __get_class_lookup(self,training_image_data):
        class_labels = [None]*training_image_data.num_classes
        class_names = training_image_data.class_indices.keys()
        for classname in class_names:
            class_labels[training_image_data.class_indices[classname]] = classname
        return class_labels
    def __define_model(self):
        #print("MLFORKIDS: Defining the layers to include in your neural network")
        model = Sequential([
            # input layer is resizing all images to save having to do that in a manual pre-processing step
            Rescaling(1/127, input_shape=(224, 224, 3)),
            # using an existing pre-trained model as an untrainable main layer
            hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/classification/4"),
            #
            Dropout(rate=0.2),
            #
            Dense(self.num_classes)
        ])
        model.build((None,) + (224, 224, 3))

        # model compile parameters copied from tutorial at https://www.tensorflow.org/hub/tutorials/tf2_image_retraining
        model.compile(
            optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9),
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
            metrics=['accuracy'])

        return model
        
    def train_model(self):
        trainingimagesdata = ImageDataGenerator().flow_from_directory("./dataset",target_size = (224,224))
        self.num_classes = trainingimagesdata.num_classes
        self.ml_class_names = self.__get_class_lookup(trainingimagesdata)
        self.ml_model = self.__define_model()
        if trainingimagesdata.batch_size > trainingimagesdata.samples:
            trainingimagesdata.batch_size = trainingimagesdata.samples
        steps_per_epoch = trainingimagesdata.samples // trainingimagesdata.batch_size
        epochs = 8
        if trainingimagesdata.samples > 55:
            epochs = 30
            #epochs = 1
        self.ml_model.fit(trainingimagesdata, epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=2)
        print("MLFORKIDS: Model training complete")
    def prediction(self, image_location: str):
        if hasattr(self, "ml_model") == False:
            raise RuntimeError("Machine learning model has not been trained for this project")
        testimg = image.load_img(image_location, target_size=(224,224))
        testimg = image.img_to_array(testimg)
        testimg = np.expand_dims(testimg, axis=0)
        predictions = self.ml_model.predict(testimg)
        topprediction = predictions[0]
        topanswer = np.argmax(topprediction)
        return self.ml_class_names[topanswer]
            
    def md_save(self):
        self.ml_model.save("mymodel")
    def md_load(self):
        self.ml_model = keras.models.load_model("mymodel")
        

In [4]:
rp = roopre()

In [5]:
rp.train_model()

Found 2001 images belonging to 20 classes.


/Users/roopre/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/30
62/62 - 44s - loss: 2.6300 - accuracy: 0.4093
Epoch 2/30
62/62 - 39s - loss: 1.6484 - accuracy: 0.6790
Epoch 3/30
62/62 - 39s - loss: 1.3930 - accuracy: 0.7547
Epoch 4/30
62/62 - 38s - loss: 1.3217 - accuracy: 0.7867
Epoch 5/30
62/62 - 38s - loss: 1.1939 - accuracy: 0.8268
Epoch 6/30
62/62 - 38s - loss: 1.1455 - accuracy: 0.8542
Epoch 7/30
62/62 - 38s - loss: 1.1123 - accuracy: 0.8664
Epoch 8/30
62/62 - 38s - loss: 1.0731 - accuracy: 0.8735
Epoch 9/30
62/62 - 38s - loss: 1.0475 - accuracy: 0.8949
Epoch 10/30
62/62 - 39s - loss: 1.0160 - accuracy: 0.9005
Epoch 11/30
62/62 - 42s - loss: 1.0129 - accuracy: 0.9032
Epoch 12/30
62/62 - 40s - loss: 0.9892 - accuracy: 0.9198
Epoch 13/30
62/62 - 38s - loss: 0.9810 - accuracy: 0.9203
Epoch 14/30
62/62 - 38s - loss: 0.9612 - accuracy: 0.9264
Epoch 15/30
62/62 - 39s - loss: 0.9716 - accuracy: 0.9259
Epoch 16/30
62/62 - 38s - loss: 0.9587 - accuracy: 0.9411
Epoch 17/30
62/62 - 38s - loss: 0.9526 - accuracy: 0.9330
Epoch 18/30
62/62 - 38s

KeyboardInterrupt: 

In [6]:
rp.md_save()

In [114]:
rp.md_load()

In [103]:
rp.prediction("test.jpeg")

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


{'class_name': '돈까스', 'confidence': 96.54781222343445}